In [ ]:
!pip install -U tensorflow-text

In [ ]:
!git clone -l -s https://github.com/Krinistopen/hse-workshop.git
%cd hse-workshop
!ls

In [37]:
# import tensorflow_hub as hub
# import tensorflow as tf
import numpy as np
# import tensorflow_text
# from google.colab import files
import pandas as pd

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prfs

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
data = pd.read_csv('data.csv', sep=';')
text_var = 'text'
class_var = 'mark'

In [ ]:
data.head() 

In [ ]:
df = data[class_var].value_counts(normalize=True) * 100
df.plot.bar(x=class_var)

In [ ]:
data = data[(data[class_var] != -1)]

In [ ]:
marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new
del marks_new

In [ ]:
train_vol = 0.7

In [ ]:
train, test = train_test_split(data, train_size = train_vol, random_state = 99, stratify = data[class_var])
train = train.drop_duplicates(subset=[text_var])

In [ ]:
check = []
train_data = train[text_var].tolist()
for text in test[text_var]:
    if text not in train_data:
        check.append('KEEP')
    else:
        check.append('IGNORE')
test['check'] = check
test = test.loc[test['check'] == 'KEEP']
test = test.drop(['check'], axis=1)
del check

In [ ]:
train_embeddings = []
for text in train_data:
    embedding = embed(str(text))[0]
    train_embeddings.append(embedding)

In [ ]:
test_embeddings = []
for text in test[text_var]:
    embedding = embed(str(text))[0]
    test_embeddings.append(embedding)

In [ ]:
# nbg = GaussianNB()
# nbb = BernoulliNB()
# lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)
svm = SVC(kernel='poly', gamma='scale', probability=True)

In [ ]:
classes = train[class_var].tolist()
classes_to_check = test[class_var].tolist()

In [ ]:
# nbg.fit(train_embeddings,classes)
# nbb.fit(train_embeddings,classes)
# lg.fit(train_embeddings,classes)
svm.fit(train_embeddings,classes)

In [ ]:
# classesNBG = nbg.predict(test_embeddings)
# classesNBB = nbb.predict(test_embeddings)
# classesLG = lg.predict(test_embeddings)
classesSVM = svm.predict(test_embeddings)

In [ ]:
metrics = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')

In [ ]:
metrics

### То же самое, только с SVD векторами

In [10]:
import gensim
import pandas as pd
import pymorphy2
import pprint
from collections import defaultdict
from gensim import corpora, models
import re

In [85]:
morph = pymorphy2.MorphAnalyzer()
frequency = defaultdict(int)
from rustoplist import stopwords

In [5]:
def tokenize(document):
    '''
    Токенизатор. Фильтрует текст от пунктуации, цифр, если они не являются частью слова 
    (чтобы оставить токены типа "ТЕЛЕ2", "2gis" и т.п), множественных пробелов.
    ''' 
    punc_1 = re.compile(r'[\.,;:\!\?/\|\\@#\$%\^\&\*)(_=\+\]\[}{"`~<>»«\'@]')
    punc_2 = re.compile(r'[،;؛¿!"\])}»›”؟¡%٪°±©®।॥…“‘„‚«‹「『\–—]')
    document = document.lower()
    document = re.sub(punc_1, ' ', document)
    document = re.sub(punc_2, ' ', document)
    document = re.sub(r'(?<!\w)\d+|\d+(?!\w)', ' ', document)
    document = re.sub(r'\s+', ' ', document)
    for token in document.strip().split():
        yield token

In [6]:
def morphanalyze(token):
    '''
    Лемматизатор. Использует морфоанализатор pymorphy2. 
    Для токенов не из словаря формирует гипотезы и возвращает 
    наиболее вероятную.
    '''
    hypothesis = morph.parse(token)[0]
    lemma = hypothesis.normal_form
    return lemma

In [7]:
def applyCutoff(texts, cutoff):
    '''
    Удаление редких слов. Порог задается вручную.
    '''
    for text in texts:
        for token in text:
            frequency[token] += 1
    new_texts = [[token for token in text if frequency[token] >= cutoff]
                for text in texts]
    return new_texts

In [8]:
def preproc(documents, cutoff=2, stoplist=stopwords):
    '''
    Подготовка текстовых данных к извлечению тематик.
    '''
    texts = []

    if stoplist is not None:
        for document in documents:
            text = []
            for token in tokenize(document):
                token = morphanalyze(token)
                if token not in stoplist:
                    text.append(token)
            texts.append(text)
        new_texts = applyCutoff(texts, cutoff)

    else:
        for document in documents:
            text = []
            for token in tokenize(document):
                token = morphanalyze(token)
                text.append(token)
            texts.append(text)
        new_texts = applyCutoff(texts, cutoff)

    del texts
    return new_texts

In [11]:
data = pd.read_csv('data.csv', sep=';')
text_var = 'text'
class_var = 'mark'
data = data[(data[class_var] != -1)]
marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new
del marks_new

In [86]:
texts = data[text_var]
ids = data['id']

In [87]:
texts_preproc = preproc(texts)

In [98]:
dictionary = corpora.Dictionary(texts_preproc)
corpus = [dictionary.doc2bow(text) for text in texts_preproc]

In [68]:
pprint.pprint(corpus[0])

[]


In [19]:
tfidf = models.TfidfModel(corpus)
corpus_weighted = tfidf[corpus]

In [67]:
corpus_weighted[0]

[]

In [23]:
model = models.LsiModel(corpus_weighted, id2word=dictionary, num_topics=300)

In [24]:
model.show_topics()[:5]

[(0,
  '0.579*"комната" + 0.551*"тёплый" + 0.233*"скисание" + 0.208*"молоко" + 0.191*"являться" + 0.183*"явление" + 0.177*"химический" + 0.165*"вещество" + 0.135*"состав" + 0.127*"процесс"'),
 (1,
  '-0.354*"комната" + -0.334*"тёплый" + 0.289*"являться" + 0.287*"явление" + 0.287*"вещество" + 0.262*"химический" + 0.241*"состав" + 0.237*"новое" + 0.206*"процесс" + 0.206*"образоваться"'),
 (2,
  '-0.494*"свет" + -0.445*"металл" + -0.417*"излучение" + -0.396*"раскалить" + 0.246*"новое" + 0.229*"образоваться" + 0.173*"вещество" + -0.107*"радуга" + -0.094*"появление" + -0.088*"дождь"'),
 (3,
  '-0.505*"новое" + -0.473*"образоваться" + 0.424*"состав" + 0.351*"меняться" + 0.213*"изменяться" + -0.145*"вещество" + -0.131*"свет" + -0.123*"металл" + -0.112*"излучение" + -0.110*"раскалить"'),
 (4,
  '0.462*"происходить" + 0.366*"окисление" + 0.349*"реакция" + 0.311*"процесс" + -0.309*"состав" + -0.305*"меняться" + -0.184*"изменяться" + -0.169*"вещество" + 0.147*"химический" + -0.145*"новое"')]

In [66]:
model[corpus_weighted[0]]

[]

In [63]:
embeddings = {}

for i, ID in enumerate(ids):
    embedding_raw = model[corpus_weighted[i]]
    embedding = [value for ind, value in embedding_raw]
    embeddings.update({ID: embedding})

644
976
2114
2196
2433
2473


In [38]:
train_vol = 0.7
train, test = train_test_split(data, train_size = train_vol, random_state = 99, stratify = data[class_var])
train = train.drop_duplicates(subset=[text_var])

In [39]:
check = []
train_data = train[text_var].tolist()
for text in test[text_var]:
    if text not in train_data:
        check.append('KEEP')
    else:
        check.append('IGNORE')
test['check'] = check
test = test.loc[test['check'] == 'KEEP']
test = test.drop(['check'], axis=1)
del check

C:\Users\ruskdu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
train_embeddings = [embeddings[ID] for ID in train['id'].tolist()]
test_embeddings = [embeddings[ID] for ID in test['id'].tolist()]

In [42]:
svm = SVC(kernel='poly', gamma='scale', probability=True)

In [44]:
classes = train[class_var].tolist()
classes_to_check = test[class_var].tolist()

In [49]:
svm.fit(train_embeddings,classes)

ValueError: setting an array element with a sequence.

### Код для проверки среднего результата на 10 прогонах

In [ ]:
df_results = pd.DataFrame(columns=['classifier', 'train_size', 'average_precision',
                                   'average_recall', 'average_fscore'])
precision_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}
recall_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}
fscore_all = {'NBG':[], 'NBB':[], 'SVM':[], 'LG':[]}

data = pd.read_excel('/opt/projects/docs_to_vecs/sample_data/mcko_random_sample_C1_9602.xlsx')

text_var = 'info_new'
class_var = 'mark'

data = data.loc[(data[class_var] != -1)]

marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new

train_vol = 0.7

for i in range(10):
    train, tmp = train_test_split(data, train_size = train_vol, stratify = data[class_var])
    train = train.drop_duplicates(subset=[text_var])
    test = data
    check = []
    train_data = train[text_var].tolist()
    for text in data[text_var]:
        if text not in train_data:
            check.append('KEEP')
        else:
            check.append('IGNORE')
    test['check'] = check
    test = test.loc[test['check'] == 'KEEP']

    train_embeddings = []
    for text in train_data:
        embedding = embed(str(text))[0]
        train_embeddings.append(embedding)
    print('Прогон {}, вектора для обучения готовы.'.format(i+1))

    test_embeddings = []
    for text in test[text_var]:
        embedding = embed(str(text))[0]
        test_embeddings.append(embedding)
    print('Прогон {}, вектора для тестирования готовы.'.format(i+1))

    nbg = GaussianNB()
    nbb = BernoulliNB()
    svm = SVC(kernel='poly', gamma='scale', probability=True)
    lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)

    classes = train[class_var].tolist()

    nbg.fit(train_embeddings,classes)
    nbb.fit(train_embeddings,classes)
    svm.fit(train_embeddings,classes)
    lg.fit(train_embeddings,classes)
    print('Прогон {}, все модели обучены.'.format(i+1))

    target = 0

    classesNBG = nbg.predict(test_embeddings)
    scoresNBG = [score[target] for score in nbg.predict_proba(test_embeddings)]
    classesNBB = nbb.predict(test_embeddings)
    scoresNBB = [score[target] for score in nbb.predict_proba(test_embeddings)]
    classesSVM = svm.predict(test_embeddings)
    scoresSVM = [score[target] for score in svm.predict_proba(test_embeddings)]
    classesLG = lg.predict(test_embeddings)
    scoresLG = [score[target] for score in lg.predict_proba(test_embeddings)]

    classes_to_check = test[class_var].tolist()

    precisionNBG, recallNBG, fscoreNBG = prfs(classes_to_check, classesNBG, pos_label = 0, average = 'binary')[0:3]
    precisionNBB, recallNBB, fscoreNBB = prfs(classes_to_check, classesNBB, pos_label = 0, average = 'binary')[0:3]
    precisionSVM, recallSVM, fscoreSVM = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')[0:3]
    precisionLG, recallLG, fscoreLG = prfs(classes_to_check, classesLG, pos_label = 0, average = 'binary')[0:3]

    precision_all['NBG'].append(precisionNBG)
    precision_all['NBB'].append(precisionNBB)
    precision_all['SVM'].append(precisionSVM)
    precision_all['LG'].append(precisionLG)
    recall_all['NBG'].append(recallNBG)
    recall_all['NBB'].append(recallNBB)
    recall_all['SVM'].append(recallSVM)
    recall_all['LG'].append(recallLG)
    fscore_all['NBG'].append(fscoreNBG)
    fscore_all['NBB'].append(fscoreNBB)
    fscore_all['SVM'].append(fscoreSVM)
    fscore_all['LG'].append(fscoreLG)
    print('Прогон {} закончен.'.format(i+1))


for model in ['NBG', 'NBB', 'SVM', 'LG']:
    df_results = df_results.append({'classifier': model,
                                'average_precision':sum(precision_all[model])/10,
                                'average_recall':sum(recall_all[model])/10,
                                'average_fscore':sum(fscore_all[model])/10},
                                 ignore_index=True)

df_results.set_index('classifier')
df_results_t = df_results.transpose()
df_results_t.columns = df_results_t.iloc[0]
df_results_t.reset_index(inplace=True)
df_results_t = df_results_t.drop([0], axis=0)
df_results_t.plot.bar('index')